#  IBM Data Science Capstone Project - The Battle of Neighborhoods

<b> by Sumit Kumar Gupta <b>

-----------

## Introduction


<b>Pets</b> are like family to most us, as world is getting lonelier for some, more and more humans are finding solace
in keeping a pet in the house,as a result the pet industry is booming and is worth billion of dollors. So, this
project will analyise the pet eco-system in different neighbourhoods of <i><b>New York City</b></i> by utitlizing the 
data from <b>Foursquare</b> Places api and in doing so we can have answers to several question like which part of the city is most pet friendly,
which location will be best for my pet related bussiness,which options do i have for my relocation with my pet, do i need to make my bussiness
pet-firendly to attract more consumers ? 








-----

## Data

In [ ]:
For this project we have used the fo

 In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.link to the dataset:

https://geo.nyu.edu/catalog/nyu_2451_34572

<b>Data Analysis </b>

In [44]:
# library to handle data in a vectorized manner
import numpy as np 

# library for data analsysis
import pandas as pd 
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# library to handle JSON files
import json 

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

 # library to handle requests
import requests
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

print('Libraries imported.')

Libraries imported.


 ### New York City Neighborhood data

In [26]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [27]:
#load the data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [28]:
neighborhoods_data = newyork_data['features']

#### Tranform the data into a *pandas* dataframe

In [29]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [30]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [31]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Use geopy library to get the latitude and longitude values of New York City.

In [45]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [46]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [19]:
#### Define Foursquare Credentials and Version

In [47]:
# The code was removed by Watson Studio for sharing.

In [48]:

LIMIT = 100 
radius = 1000 


<b>From the forusquare documentation , we have shortlisted the categories we are interested in to get our results  </b>

In [50]:
# getting the categories of the data we are interested in 

pet_Cafe = '56aa371be4b08b9a8d573508'
pet_service = '5032897c91d4c4b30a586d69'
petstore = '4bf58dd8d48988d100951735'
vet =  '4d954af4a243a5684765b473'
dog_run = '4bf58dd8d48988d1e5941735'


<b>Defining a function to get nearby results for pet in  each neighbourhood</b>

In [ ]:
def getNearbyVenues(borough,names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
     
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&time={}&day={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude,
            'any',
            'any',
            radius, 
            LIMIT,
            pet_Cafe+','+pet_service+','+petstore+','+vet+','+dog_run
            )
        
        if(requests.get(url).json()["response"]['groups'][0]['items']):
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough','Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [22]:
neighborhoods.shape

(306, 4)

we are going to analyzie each neighbourhood of new city in each borough for pet friendliness and pet related services , in order to do that efficently we need to atuomate a lot of things 